In [ ]:
import pandas as pd
df1  =pd.read_csv('/content/drive/MyDrive/校创/娟低高1.csv').astype(str)
df2 = pd.read_csv('/content/drive/MyDrive/校创/其他类.csv').astype(str)
df2['label'] = 2

In [ ]:
df = pd.concat([df1,df2])

In [ ]:
df

,Unnamed: 0,label,content
0,1,0,﻿ 第 章全 不知 幻想 一个 童话国 里 住 男孩儿 女孩儿 丁点儿 男孩儿 女孩儿 叫做...
1,2,0,﻿ 第一章 万事通 战胜 小星星 教授 全 不知 太阳城 旅行 回来 已经 过去 两年 半 ...
2,3,0,﻿ 第 章 花城 矮子 一座 童话 城市 里 住 矮子 管 叫 矮子 是因为 很小 每个 矮...
3,4,0,一个 贵族 女儿 风儿 草 吹 过去 时候 田野 就 像 湖水 一起 涟漪 麦子 扫...
4,5,0,一朵 叫 童话 蒲公英 斑马 一起 奔跑 一个 懒洋洋 季节 茉莉 正 坐在 树 发呆 一匹...
...,...,...,...
322,323,2,本书 wwwabadacn 免费 txt 小说 下载站 更新 免费 电子书 关注 wwwab...
323,324,2,相关 书评 文化 簇拥 历史 命运 藏族 小说 首席 金座 活佛 海杰 藏族 文化 成为 热...
324,325,2,驼峰 航线 自序 驼峰 航线 绵延 千里 终年 白雪皑皑 喜马拉雅山 山脉 横断山脉 上空 ...
325,326,2,骚戏 在此之前 细读 西门 两部 长篇小说 敏感 莲衣 现实 张扬 躁气 讲述 历史 香艳 ...


In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd
import jieba
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
tokenizer=Tokenizer()
tokenizer.fit_on_texts(df['content'])
vocab=tokenizer.word_index #得到每个词的编号
x_train, x_test, y_train, y_test = train_test_split(df['content'], df['label'], test_size=0.2)
x_train_word_ids=tokenizer.texts_to_sequences(x_train)
x_test_word_ids = tokenizer.texts_to_sequences(x_test)

In [ ]:
from gensim.models import keyedvectors
import gensim
import numpy as np
#w2v=keyedvectors.load_word2vec_format(os.path.join(root_path,"wiki_word2vec_50.bin"),binary=True)
w2v_model = gensim.models.KeyedVectors.load_word2vec_format("/content/drive/MyDrive/校创/wiki_word2vec_50.bin",binary=True)
embedding_matrix = np.zeros((len(vocab) + 1, 50))
for word, i in vocab.items():
    try:
        embedding_vector = w2v_model[str(word)]
        embedding_matrix[i] = embedding_vector
        if word=='妈妈':
            print(embedding_vector)
    except KeyError:
        continue

[ 0.9131416   0.4671083   0.2711738   0.33765495  0.43602702  0.18306577
 -0.6726549   0.66305834  0.4089764   0.1696789  -0.28275257 -0.32089975
  0.347405    0.16419958 -0.08157358 -0.2560843   0.30017748  0.02375275
 -0.05284187  0.17037767  0.10020204  0.47761682 -0.670955    0.91816175
  0.31028727 -0.7130477  -0.57554674 -0.25559598 -0.11245019 -0.3867515
  0.3385001   0.47431403  0.5860103   0.37273115  0.26878178 -0.4529409
  0.28356767 -0.00861329 -0.36427543 -0.37893915 -0.53132355 -0.3122926
 -0.00731339  0.69051987 -0.10986451  0.15745394  0.23978733 -0.32519662
  0.26006478  0.04614682]


In [ ]:
from keras.layers.merge import concatenate
from keras.models import Model
from keras import  Input
import keras
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Embedding,Conv1D,MaxPooling1D,Flatten,Dense,Dropout,BatchNormalization
import sklearn.metrics as metrics
def TextCNN_model_2(x_train_padded_seqs,y_train,x_test_padded_seqs,y_test,embedding_matrix,shape1):
    # 模型结构：词嵌入-卷积池化*3-拼接-全连接-dropout-全连接
    main_input = Input(shape=(shape1,), dtype='float64')
    # 词嵌入（使用预训练的词向量）
    embedder = Embedding(len(vocab) + 1, 50, input_length=shape1, weights=[embedding_matrix], trainable=False)
    embed = embedder(main_input)
    # 词窗大小分别为3,4,5
    cnn1 = Conv1D(256, 3, padding='same', strides=1, activation='relu')(embed)
    cnn1 = MaxPooling1D(pool_size=38)(cnn1)
    cnn2 = Conv1D(256, 4, padding='same', strides=1, activation='relu')(embed)
    cnn2 = MaxPooling1D(pool_size=37)(cnn2)
    cnn3 = Conv1D(256, 5, padding='same', strides=1, activation='relu')(embed)
    cnn3 = MaxPooling1D(pool_size=36)(cnn3)
    # 合并三个模型的输出向量
    cnn = concatenate([cnn1, cnn2, cnn3], axis=1)
    flat = Flatten()(cnn)
    drop = Dropout(0.4)(flat)
    main_output = Dense(3, activation='softmax')(drop)
    model = Model(inputs=main_input, outputs=main_output)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    one_hot_labels = np_utils.to_categorical(y_train, num_classes=3)  # 将标签转换为one-hot编码
    model.fit(x_train_padded_seqs, one_hot_labels, batch_size=32, epochs=5)
    #y_test_onehot = keras.utils.to_categorical(y_test, num_classes=3)  # 将标签转换为one-hot编码
    result = model.predict(x_test_padded_seqs)  # 预测样本属于每个类别的概率
    result_labels = np.argmax(result, axis=1)  # 获得最大概率对应的标签
    result_labels = list(result_labels)
    y_predict = list(map(str, result_labels))
    y_pred = list(result_labels)
    t_test = list(y_test)
    print(y_pred)
    print('准确率', metrics.accuracy_score(t_test, y_predict))
    print('平均f1-score:', metrics.f1_score(t_test, y_predict, average='weighted'))
    return metrics.accuracy_score(t_test, y_predict)
#TextCNN_model_2(x_train_padded_seqs,y_train,x_test_padded_seqs,y_test,embedding_matrix,)


In [ ]:
p = 50
a = []
for i in range(100):
    x_train_padded_seqs=pad_sequences(x_train_word_ids,maxlen=p) #将超过固定值的部分截掉，不足的在最前面用0填充
    x_test_padded_seqs=pad_sequences(x_test_word_ids, maxlen=p)
    #x_val_padded_seqs = pad_sequences(x_val_word_ids, maxlen=p)
    print("词汇量",p)
    a.append(TextCNN_model_2(x_train_padded_seqs,y_train,x_test_padded_seqs,y_test,embedding_matrix,p))
    p+=50

词汇量 50
Epoch 1/5
25/25 [==============================] - 2s 33ms/step - loss: 1.1526 - accuracy: 0.3919
Epoch 2/5
25/25 [==============================] - 1s 33ms/step - loss: 0.8443 - accuracy: 0.6120
Epoch 3/5
25/25 [==============================] - 1s 33ms/step - loss: 0.7166 - accuracy: 0.6883
Epoch 4/5
25/25 [==============================] - 1s 33ms/step - loss: 0.5881 - accuracy: 0.7964
Epoch 5/5
25/25 [==============================] - 1s 32ms/step - loss: 0.4783 - accuracy: 0.8740
[0, 0, 0, 2, 2, 2, 0, 0, 2, 0, 0, 1, 0, 2, 2, 0, 0, 2, 2, 0, 0, 2, 0, 2, 2, 2, 1, 1, 1, 2, 0, 0, 0, 2, 0, 1, 1, 2, 0, 2, 0, 0, 2, 2, 2, 1, 1, 0, 2, 2, 2, 2, 2, 0, 0, 0, 2, 0, 2, 2, 2, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 2, 0, 2, 2, 2, 2, 0, 1, 1, 2, 2, 0, 2, 2, 2, 0, 2, 2, 0, 2, 0, 0, 0, 1, 1, 1, 2, 1, 2, 1, 2, 0, 0, 2, 1, 2, 0, 2, 2, 1, 0, 0, 2, 2, 2, 0, 0, 1, 0, 2, 0, 0, 2, 2, 2, 1, 0, 2, 1, 0, 2, 0, 0, 2, 1, 0, 1, 2, 2, 0, 1, 2, 0, 0, 0, 0, 1, 0, 2, 1, 1, 0, 1, 0, 2, 2, 0, 2, 1, 0, 0, 0, 0